<a href="https://colab.research.google.com/github/p-disha/ShopUNow-Agent/blob/main/ShopUNow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####################################


##Tech Stack

LangChain / langchain_community-	Provides VectorStores (FAISS), Document abstraction, Embeddings, and Retrieval.
FAISS (vectorstore)-	For embedding storage & similarity search (RAG).
Sentence-Transformers embeddings-	To convert document chunks into embedding vectors.
**pdfminer.six + pytesseract + PIL**-	Extract text from PDFs, images (OCR) and markdown/text files — for building corpus.
Markdownify	Convert markdown files to plain text.
LangGraph (StateGraph etc.)-	The agent orchestration framework: state + nodes + transitions.
Pydantic-	For structured schemas of state and tool inputs (validation, typing).
LLM backends- OpenAI, Gemini (if available)	For synthesis / general LLM responses.

**Parses different document types (text, csv, pdf, image) into a corpus.**

**Chunks documents into manageable pieces using RecursiveCharacterTextSplitter.**

**Builds a FAISS index, persists it.**

**Sets up intent routing + tools for order status, returns, tickets.**

**Handles RAG retrieval + LLM synthesis with system prompt.**

**Passes retriever via RunnableConfig/configurable, avoiding earlier bug.**

**Good structure using StateGraph, Pydantic state schemas.**

# Task
Modify the code to use Gemini as the primary LLM and fallback to OpenAI, and add tools for order status, returns, and tickets.

## Integrate llms (gemini and openai)

### Subtask:
Modify the agent to use a language model for answer synthesis, with Gemini as the primary and OpenAI as a fallback.


**Reasoning**:
Define the `get_chat_model` function to handle primary (Gemini) and fallback (OpenAI) LLM initialization and then update the `synthesis_node` to use this LLM for answer generation based on context and user input.



**Reasoning**:
The previous command failed because `langchain_google_genai` was not installed. Install the missing package and try the imports and function definition again.



In [3]:
# =========================
# Cell A: Setup and Vector Store + LLM
# =========================

# Install required packages (include Google-GenAI integration if using Gemini)
!pip install -qU langchain_community faiss-cpu langchain_openai langchain-google-genai pydantic typing_extensions vaderSentiment langgraph

!pip install -qU flask flask-cors pyngrok


# Imports
import os
from typing import List, Dict, Any, Optional, Literal
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage

# Make sure API keys are set
try:
    from google.colab import userdata
    key = userdata.get("OPENAI_API_KEY")
    if key:
        os.environ["OPENAI_API_KEY"] = key
    gem_key = userdata.get("GEMINI_API_KEY")
    if gem_key:
        os.environ["GOOGLE_API_KEY"] = gem_key
except Exception:
    pass

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GEMINI_API_KEY = os.environ.get("GOOGLE_API_KEY")  # for Gemini

assert OPENAI_API_KEY or GEMINI_API_KEY, "Please set OPENAI_API_KEY or GEMINI_API_KEY in environment variables."

# Initialize embeddings (use OpenAI embeddings; you can use Gemini embeddings if you want and have the key)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Sample documents / FAQ data with department metadata
# You should expand these to 10-15 QA per department later
faq_docs = [
    Document(page_content="Support hours are 9 AM–9 PM IST, Monday to Saturday", metadata={"department": "Customer Support"}),
    Document(page_content="How to contact support email or phone", metadata={"department": "Customer Support"}),
    Document(page_content="Return window is 10 days from delivery", metadata={"department": "Orders & Returns"}),
    Document(page_content="How can I initiate a return process", metadata={"department": "Orders & Returns"}),
    Document(page_content="We accept UPI, credit cards, wallets, and COD", metadata={"department": "Payments & Billing"}),
    Document(page_content="How to apply coupon at checkout", metadata={"department": "Payments & Billing"})
]

# Build the FAISS vector store
import faiss

# Compute embedding dimension
dim = len(embeddings.embed_query("hello world"))  # length of vector

index = faiss.IndexFlatL2(dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={}
)

ids = [f"doc{i+1}" for i in range(len(faq_docs))]
vector_store.add_documents(documents=faq_docs, ids=ids)

# Initialize LLM (Chat model)
def get_chat_model():
    if GEMINI_API_KEY:
        try:
            print("Using Gemini LLM")
            return ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
        except Exception as e:
            print("Gemini init failed:", e)
    if OPENAI_API_KEY:
        try:
            print("Using OpenAI model")
            return ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
        except Exception as e:
            print("OpenAI init failed:", e)
    # fallback mock model
    class _Mock:
        def invoke(self, messages: List[Any]):
            last_user = None
            for m in reversed(messages):
                if isinstance(m, HumanMessage):
                    last_user = m
                    break
            return type("Obj", (), {"content": "[MOCK] " + (last_user.content if last_user else "")})
    print("Using mock LLM fallback")
    return _Mock()

LLM = get_chat_model()

# Optionally define system policy / prompt template
SYSTEM_POLICY = (
    "You are ShopUNow Assistant. Be concise and accurate. Use the internal knowledge base when possible. "
    "Cite sources. If you cannot answer, ask a clarifying question."
)

print("Cell A setup complete: vector_store and LLM are initialized.")


Using Gemini LLM
Cell A setup complete: vector_store and LLM are initialized.


In [35]:
# =========================
# Cell A.1 - Load FAQ Dataset & Build Vector Store
# =========================
import json
import faiss
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# ---- Step 1: Load JSONL file ----
faq_path = "/content/shopunow_faqs.jsonl"
faq_docs = []
with open(faq_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:   # skip empty lines
            continue
        try:
            record = json.loads(line)
            question = record.get("question", "").strip()
            answer = record.get("answer", "").strip()
            dept = record.get("department", "unknown").strip()

            # Store Q + A in embeddings for better recall
            combined_text = f"Q: {question}\nA: {answer}"

            faq_docs.append(
                Document(
                    page_content=combined_text,
                    metadata={
                        "department": dept,
                        "question": question,
                        "answer": answer
                    }
                )
            )
        except json.JSONDecodeError as e:
            print(f"⚠️ Skipping bad line: {line[:80]}... | Error: {e}")

print(f"Loaded {len(faq_docs)} FAQs from {faq_path}")

# ---- Step 2: Build FAISS Vector Store ----
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

embedding_dim = len(embedding_model.embed_query("hello world"))

# Use cosine similarity (inner product) instead of L2
faiss_index = faiss.IndexFlatIP(embedding_dim)

faq_vector_store = FAISS(
    embedding_function=embedding_model,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

faq_ids = [f"faq_{i+1}" for i in range(len(faq_docs))]
faq_vector_store.add_documents(documents=faq_docs, ids=faq_ids)

print(f"✅ Vector store built with {len(faq_docs)} FAQs "
      f"across {len(set(d.metadata['department'] for d in faq_docs))} departments")


Loaded 119 FAQs from /content/shopunow_faqs.jsonl
✅ Vector store built with 119 FAQs across 5 departments


In [38]:
# =========================
# Cell B - Agent Definition with JSONL Vector Store
# (cosine similarity + dept thresholds + fuzzy fallback + deterministic seeds)
# =========================
import json
import faiss
import numpy as np
import random
from typing import Optional, List, Dict, Any, Literal
from typing_extensions import Annotated
from operator import add
from pydantic import BaseModel, Field
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from rapidfuzz import fuzz

from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# ---- Deterministic seeds ----
random.seed(42)
np.random.seed(42)

# ---- Load FAQ dataset from JSONL ----
jsonl_path = "/content/shopunow_faqs.jsonl"
faq_documents = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        record = json.loads(line)
        faq_documents.append(
            Document(
                page_content=f"Q: {record['question']}\nA: {record['answer']}",
                metadata={
                    "department": record["department"],
                    "question": record["question"],
                    "answer": record["answer"]
                }
            )
        )

print(f"Loaded {len(faq_documents)} FAQs from {jsonl_path}")

# ---- Normalized Embeddings (cosine similarity) ----
class NormalizedOpenAIEmbeddings(OpenAIEmbeddings):
    def embed_query(self, text: str) -> List[float]:
        v = np.array(super().embed_query(text))
        return (v / (np.linalg.norm(v) + 1e-10)).tolist()
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        vs = np.array(super().embed_documents(texts))
        return (vs / (np.linalg.norm(vs, axis=1, keepdims=True) + 1e-10)).tolist()

embedding_model = NormalizedOpenAIEmbeddings(model="text-embedding-ada-002")

dim = len(embedding_model.embed_query("hello world"))
faiss_index = faiss.IndexFlatIP(dim)  # cosine similarity via inner product

faq_vector_store = FAISS(
    embedding_function=embedding_model,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

faq_ids = [f"faq_{i+1}" for i in range(len(faq_documents))]
faq_vector_store.add_documents(documents=faq_documents, ids=faq_ids)

print(f"✅ Vector store ready with {len(faq_documents)} documents "
      f"across {len(set(d.metadata['department'] for d in faq_documents))} departments")

# ---- Sentiment + Dept Classifier ----
sentiment_analyzer = SentimentIntensityAnalyzer()

def detect_sentiment(text: str) -> Literal["negative","neutral","positive"]:
    if not text:
        return "neutral"
    scores = sentiment_analyzer.polarity_scores(text)
    compound = scores.get("compound", 0.0)
    if compound <= -0.3: return "negative"
    if compound >= 0.3: return "positive"
    return "neutral"

def classify_department(user_query: str) -> Optional[str]:
    query = (user_query or "").lower()
    if any(kw in query for kw in ["order status", "track order", "where is my order", "order tracking", "delayed"]):
        return "Orders & Returns"
    if any(kw in query for kw in ["return", "refund", "replace", "exchange"]):
        return "Orders & Returns"
    if any(kw in query for kw in ["payment", "upi", "card", "wallet", "cod", "invoice", "coupon"]):
        return "Payments & Billing"
    if any(kw in query for kw in ["support", "contact", "help", "issue", "complaint", "hours"]):
        return "Customer Support"
    if any(kw in query for kw in ["password", "vpn", "access", "onboarding", "hardware",
                                  "software", "leave", "policy"]):
        return "HR & IT Helpdesk"
    return None

# ---- Dept-specific similarity thresholds ----
DEPT_THRESHOLDS = {
    "Orders & Returns": 0.80,
    "Payments & Billing": 0.78,
    "Customer Support": 0.75,
    "HR & IT Helpdesk": 0.70,
    None: 0.76  # default
}

# ---- Agent State ----
class AgentState(BaseModel):
    user_input: str
    department: Optional[str] = None
    sentiment: Optional[Literal["negative","neutral","positive"]] = None
    tools_used: Annotated[List[str], add] = Field(default_factory=list)
    retrieved: Annotated[List[Dict[str, Any]], add] = Field(default_factory=list)
    intent: Optional[Literal["rag","order_status","return_create",
                             "ticket","human_escalation","unknown"]] = None
    answer: Optional[str] = None

# ---- Routing ----
def route_intent(state: AgentState) -> Dict[str, Any]:
    user_query = state.user_input
    query_lower = (user_query or "").lower()
    sentiment = detect_sentiment(user_query)
    predicted_department = classify_department(user_query)

    # Negative sentiment → escalate
    if sentiment == "negative":
        intent = "human_escalation"
    # Explicit rules
    elif any(kw in query_lower for kw in ["order status", "track order", "where is my order", "order tracking"]):
        intent = "order_status"
    elif "delayed" in query_lower and "return" not in query_lower:
        intent = "order_status"
    elif any(kw in query_lower for kw in ["return", "refund", "replace", "exchange"]):
        # If asking about return *policy* or days → informational
        if "policy" in query_lower or "how many" in query_lower or "days" in query_lower:
            intent = "rag"
        else:
            intent = "return_create"
    elif any(kw in query_lower for kw in ["ticket", "helpdesk", "support issue", "complaint", "problem"]):
        intent = "ticket"
    else:
        intent = "rag"

    print(f"[route_intent] input={user_query!r} -> intent={intent}, dept={predicted_department}, sentiment={sentiment}")
    return {"intent": intent, "department": predicted_department, "sentiment": sentiment}

# ---- Tool Node ----
def filter_by_department(results: List[Any], predicted_department: Optional[str]) -> List[Any]:
    if not results:
        return []
    if predicted_department is None:
        return results
    filtered = [(doc, score) for doc, score in results
                if (doc.metadata or {}).get("department") == predicted_department]
    return filtered or results

def tool_node(state: AgentState) -> Dict[str, Any]:
    intent, user_query, predicted_department = state.intent, state.user_input or "", state.department
    print(f"[tool_node] intent={intent}, dept={predicted_department}, input={user_query!r}")

    # --- Direct tool responses ---
    if intent == "order_status":
        return {"answer": "Your order is being processed and will be shipped soon.",
                "tools_used": ["order_status_tool"]}
    if intent == "return_create":
        return {"answer": "Return initiated. You will receive pickup and label details via email.",
                "tools_used": ["return_create_tool"]}
    if intent == "ticket":
        return {"answer": "A support ticket has been created. Someone will get back to you shortly.",
                "tools_used": ["ticket_tool"]}
    if intent == "human_escalation":
        return {"answer": "I’m sorry for the inconvenience. Escalating to human support — "
                          "someone will reach out to you soon.",
                "tools_used": ["escalation"]}

    # --- Retrieval (RAG) ---
    if intent == "rag":
        try:
            results = faq_vector_store.similarity_search_with_score(user_query, k=5)
            results = [(doc, score) for doc, score in results if doc is not None]
            results = filter_by_department(results, predicted_department)

            if results:
                top_doc, cosine_similarity = results[0]
                print(f"[tool_node] Top cosine similarity={cosine_similarity:.4f}")
                threshold = DEPT_THRESHOLDS.get(predicted_department, DEPT_THRESHOLDS[None])

                # Below threshold → try fuzzy match
                if cosine_similarity < threshold:
                    best_doc, best_fuzzy_score = None, 0
                    for doc in faq_documents:
                        fuzzy_score = fuzz.partial_ratio(user_query.lower(), doc.metadata["question"].lower()) / 100.0
                        if fuzzy_score > best_fuzzy_score:
                            best_doc, best_fuzzy_score = doc, fuzzy_score
                    if best_doc and best_fuzzy_score >= 0.90:
                        dept_meta = best_doc.metadata.get("department", "unknown")
                        answer_text = best_doc.metadata.get("answer", best_doc.page_content)
                        return {"answer": f"{answer_text} (Dept: {dept_meta})",
                                "tools_used": ["rag_fuzzy_fallback"],
                                "retrieved": [{"question": best_doc.metadata.get("question", ""),
                                               "answer": answer_text,
                                               "fuzzy_score": float(best_fuzzy_score),
                                               "source": dept_meta}]}
                    return {"answer": "🤖 Sorry, I don’t know the exact answer to that. Escalating to human support.",
                            "tools_used": ["rag_retrieval"], "retrieved": []}

                # Passed threshold
                dept_meta = (top_doc.metadata or {}).get("department", "unknown")
                answer_text = top_doc.metadata.get("answer", top_doc.page_content)
                return {"answer": f"{answer_text} (Dept: {dept_meta})",
                        "tools_used": ["rag_retrieval"],
                        "retrieved": [{"question": top_doc.metadata.get("question", ""),
                                       "answer": answer_text,
                                       "similarity": float(cosine_similarity),
                                       "source": dept_meta}]}
            else:
                return {"answer": "Sorry, no relevant info found in our knowledge base.",
                        "tools_used": ["rag_retrieval"]}
        except Exception as e:
            print(f"[tool_node] ❌ Retrieval error: {e}")
            return {"answer": "Sorry, something went wrong while searching the knowledge base.",
                    "tools_used": ["rag_retrieval"]}

    return {"answer": "Could you please rephrase your request?", "tools_used": ["fallback"]}

# ---- Synthesis Node ----
def synthesis_node(state: AgentState) -> Dict[str, Any]:
    return {}

# ---- Build Graph ----
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(AgentState)
graph.add_node("route", route_intent)
graph.add_node("tool", tool_node)
graph.add_node("synth", synthesis_node)

graph.add_edge(START, "route")
graph.add_edge("route", "tool")
graph.add_edge("tool", "synth")
graph.add_edge("synth", END)

memory = MemorySaver()
app = graph.compile(checkpointer=memory)

# ---- Ask Function ----
def ask(user_query: str, thread_id: Optional[str] = None) -> str:
    if thread_id is None:
        import uuid
        thread_id = f"thread_{uuid.uuid4().hex}"
    output = app.invoke({"user_input": user_query},
                        config={"configurable": {"thread_id": thread_id}})
    return output.get("answer", "No answer generated.")


Loaded 119 FAQs from /content/shopunow_faqs.jsonl
✅ Vector store ready with 119 documents across 5 departments


In [39]:
# =========================
# Cell B.1 - Testing (Improved Output & Meaningful Names)
# =========================
import uuid
from pprint import pprint

test_queries = [
    "What are your support hours?",
    "Tell me order status for order id ORD-1234",
    "I want a return because the product is wrong",
    "My password reset isn't working, this is frustrating",
    "I submitted a complaint about a support issue",
    "How do I pay with UPI?",
    "How to apply for leaves?",
    "what is the leave policy?",
    "where is my order?",
    "my order is delayed",
    "whats the return polcy? how many days can i return the product?",
    "how many leaves can I take?",
    " how many days for rturn?",
    "my retturn is delayed",
    "Please replace my shirt size",
    "I need help,",  # ambiguous
]

for user_query in test_queries:
    # Create a fresh conversation ID for isolation
    conversation_id = f"conv_{uuid.uuid4().hex}"
    state = AgentState(user_input=user_query)

    # Route intent & department
    route_info = route_intent(state)

    # Run through the graph (ask function)
    response = app.invoke(
        {"user_input": user_query},
        config={"configurable": {"thread_id": conversation_id}}
    )

    # Print structured logs
    print("=" * 80)
    print(f"🧑 User Query: {user_query}")
    print(f"➡️ Intent: {route_info['intent']} | Dept: {route_info['department']} | Sentiment: {route_info['sentiment']}")
    print(f"🤖 Agent Answer: {response.get('answer', '⚠️ No answer')}")
    print(f"🛠️ Tools Used: {response.get('tools_used')}")
    if response.get("retrieved"):
        print("📄 Retrieved Context:")
        pprint(response["retrieved"])
    print("=" * 80 + "\n")


[route_intent] input='What are your support hours?' -> intent=rag, dept=Customer Support, sentiment=positive
[route_intent] input='What are your support hours?' -> intent=rag, dept=Customer Support, sentiment=positive
[tool_node] intent=rag, dept=Customer Support, input='What are your support hours?'
[tool_node] Top cosine similarity=0.8337
🧑 User Query: What are your support hours?
➡️ Intent: rag | Dept: Customer Support | Sentiment: positive
🤖 Agent Answer: Yes, live chat is available from 9 AM to 9 PM IST on our website and mobile app. (Dept: Customer Support)
🛠️ Tools Used: ['rag_retrieval']
📄 Retrieved Context:
[{'answer': 'Yes, live chat is available from 9 AM to 9 PM IST on our website '
            'and mobile app.',
  'question': 'Do you have live chat support?',
  'similarity': 0.8336710929870605,
  'source': 'Customer Support'}]

[route_intent] input='Tell me order status for order id ORD-1234' -> intent=order_status, dept=Orders & Returns, sentiment=neutral
[route_intent] i

In [12]:
# =========================
# Cell C - Flask API with ngrok (collision-safe, deep debug, auto-reuse, frontend-sync)
# =========================
import os, sys, traceback, threading, uuid, socket
from flask import Flask, request, jsonify
from flask_cors import CORS

# --------- Flask Setup (separate var from LangGraph 'app') ---------
flask_app = Flask(__name__)
CORS(flask_app)

def _debug(msg: str):
    print(msg, flush=True)

# --------- Agent Caller ---------
def call_agent(query: str) -> str:
    """Call either ask() or a compiled LangGraph app if available."""
    if "ask" in globals() and callable(globals()["ask"]):
        _debug("[AGENT] Using ask()")
        return globals()["ask"](query)

    for name in ["agent_app", "graph_app", "app"]:
        obj = globals().get(name)
        if hasattr(obj, "invoke"):
            _debug(f"[AGENT] Using graph '{name}'.invoke()")
            cfg = {"configurable": {"thread_id": f"api-{uuid.uuid4().hex}"}}
            out = obj.invoke({"user_input": query}, config=cfg)
            return out.get("answer", "No answer generated.")

    raise RuntimeError("❌ No agent available. Run Cell B first.")

# --------- Routes ---------
@flask_app.route("/ask", methods=["POST", "GET"])
def ask_api():
    try:
        _debug("\n[API] ▶️ Received /ask")
        if request.method == "POST":
            if not request.is_json:
                return jsonify({"error": "Content-Type must be application/json"}), 400
            data = request.get_json(force=True, silent=True) or {}
            query = (data.get("query") or "").strip()
        else:
            query = (request.args.get("query") or "").strip()

        if not query:
            return jsonify({"error": "Empty query"}), 400

        _debug(f"[API] Query: {query!r}")
        answer = call_agent(query)
        _debug(f"[API] ✅ Answer: {answer!r}")
        return jsonify({"query": query, "answer": answer})

    except Exception as e:
        _debug("[API] ❌ Internal server error")
        traceback.print_exc(file=sys.stdout)
        return jsonify({"error": "Internal server error", "details": str(e)}), 500

@flask_app.route("/", methods=["GET"])
def home():
    return jsonify({"status": "ok", "message": "ShopUNow Agent API is running!"})

# --------- Run Flask (dynamic port, avoids collisions) ---------
def find_free_port(default=5000):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]

PORT = find_free_port(5000)
_debug(f"▶️ Starting Flask server on port {PORT}...")

def run_flask():
    try:
        flask_app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)
    except Exception as e:
        _debug(f"❌ Flask crashed: {e}")
        traceback.print_exc(file=sys.stdout)

threading.Thread(target=run_flask, daemon=True).start()

# --------- ngrok setup ---------
try:
    from pyngrok import ngrok
except ImportError:
    _debug("[NGROK] Installing pyngrok...")
    import subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyngrok"], check=True)
    from pyngrok import ngrok

def _get_secret(name):
    try:
        from google.colab import userdata
        return userdata.get(name)
    except Exception:
        return os.getenv(name)

NGROK_AUTH_TOKEN = _get_secret("NGROK_AUTH_TOKEN")
if NGROK_AUTH_TOKEN:
    _debug("🔑 Setting ngrok auth token")
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
else:
    _debug("⚠️ No ngrok auth token found; using free session")

# Kill old tunnels before opening a new one
ngrok.kill()

try:
    _debug(f"🌐 Starting ngrok tunnel on :{PORT} ...")
    tunnel = ngrok.connect(PORT)
    public_url = getattr(tunnel, "public_url", str(tunnel))
    _debug(f"🚀 Public API URL: {public_url}")

    # ---- Write URL for frontend auto-detection ----
    backend_file = "/content/backend_url.txt"
    with open(backend_file, "w") as f:
        f.write(public_url.strip())
    _debug(f"📂 Saved backend URL to {backend_file}")

    # Print curl test
    print("\nTest with:")
    print(f'curl -X POST "{public_url}/ask" -H "Content-Type: application/json" -d "{{\\"query\\": \\"What are your support hours?\\"}}"')
except Exception as e:
    _debug(f"❌ ngrok connection failed: {e}")
    traceback.print_exc(file=sys.stdout)


▶️ Starting Flask server on port 50067...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:50067
 * Running on http://172.28.0.12:50067
INFO:werkzeug:Press CTRL+C to quit


🔑 Setting ngrok auth token
🌐 Starting ngrok tunnel on :50067 ...
🚀 Public API URL: https://c0792a802d11.ngrok-free.app
📂 Saved backend URL to /content/backend_url.txt

Test with:
curl -X POST "https://c0792a802d11.ngrok-free.app/ask" -H "Content-Type: application/json" -d "{\"query\": \"What are your support hours?\"}"


In [13]:
# =========================
# Cell D - Streamlit Frontend (Auto-detect backend, Mandatory Keys, LangSmith optional)
# =========================
import subprocess, threading, os

!pip install -q streamlit requests pyngrok

# Try to auto-read backend URL from file
backend_url_file = "/content/backend_url.txt"
default_api_url = "http://127.0.0.1:5000/ask"
if os.path.exists(backend_url_file):
    try:
        with open(backend_url_file, "r") as f:
            url = f.read().strip()
            if url:
                default_api_url = url.rstrip("/") + "/ask"
    except Exception as e:
        print("⚠️ Could not read backend_url.txt:", e)

with open("app_frontend.py", "w") as f:
    f.write(f"""
import streamlit as st
import requests

st.set_page_config(page_title="ShopUNow Agent", page_icon="🛍️", layout="centered")
st.title("🛍️ ShopUNow AI Assistant")

# --- Sidebar: Configuration ---
st.sidebar.header("🔑 Configuration (Required)")

api_url = st.sidebar.text_input("Flask API URL (auto-detected from backend)", value="{default_api_url}")
openai_key = st.sidebar.text_input("OpenAI API Key", type="password")
gemini_key = st.sidebar.text_input("Gemini API Key", type="password")
ngrok_token = st.sidebar.text_input("ngrok Auth Token", type="password")

st.sidebar.header("Optional")
langsmith_key = st.sidebar.text_input("LangSmith Key (optional)", type="password")

# ---- Validation ----
errors = []
if not api_url.strip():
    errors.append("❌ Flask API URL is required.")
if not (openai_key.strip() or gemini_key.strip()):
    errors.append("❌ At least one model key (OpenAI or Gemini) is required.")
if not ngrok_token.strip():
    errors.append("❌ ngrok Auth Token is required.")

if errors:
    for e in errors:
        st.sidebar.error(e)
    st.stop()
else:
    st.sidebar.success("✅ All required configuration set")

# Store secrets
st.session_state.secrets = {{
    "API_URL": api_url.strip(),
    "OPENAI_API_KEY": openai_key.strip(),
    "GEMINI_API_KEY": gemini_key.strip(),
    "NGROK_AUTH_TOKEN": ngrok_token.strip(),
    "LANGSMITH_KEY": langsmith_key.strip(),
}}

# --- Chat Section ---
st.subheader("💬 Chat with ShopUNow Agent")

if "chat" not in st.session_state:
    st.session_state.chat = []

query = st.text_input("Type your question here:")

if st.button("Ask"):
    if query.strip():
        st.session_state.chat.append(("🧑 You", query))
        try:
            resp = requests.post(st.session_state.secrets["API_URL"], json={{"query": query}}, timeout=20)
            if resp.status_code == 200:
                data = resp.json()
                answer = data.get("answer", "⚠️ No answer returned")
            else:
                answer = f"⚠️ Error {{resp.status_code}}: {{resp.text}}"
        except Exception as e:
            answer = f"⚠️ Request failed: {{e}}"

        st.session_state.chat.append(("🤖 Agent", answer))

# --- Display chat history ---
for sender, msg in st.session_state.chat:
    st.markdown(f"**{{sender}}:** {{msg}}")
""")

# ---- Run Streamlit in background ----
def run_streamlit():
    subprocess.run(
        ["streamlit", "run", "app_frontend.py", "--server.port", "8501", "--server.headless", "true"]
    )

threading.Thread(target=run_streamlit, daemon=True).start()

# ---- ngrok tunnel for frontend ----
from pyngrok import ngrok
print("🌐 Starting ngrok tunnel for Streamlit...")
frontend_url = ngrok.connect(8501)
print("🚀 Streamlit Frontend URL:", frontend_url)


🌐 Starting ngrok tunnel for Streamlit...
🚀 Streamlit Frontend URL: NgrokTunnel: "https://33384fb0fead.ngrok-free.app" -> "http://localhost:8501"
